In [7]:
import run_hp_tuning
import pandas as pd
import numpy as np

In [10]:
import sqlite3
conn = sqlite3.connect("results/optuna_results.db")
print(pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn))
data_merged = pd.read_sql_query("SELECT * FROM trial_values val LEFT JOIN trial_params par on val.trial_id = par.trial_id", conn)

                         name
0                     studies
1                version_info
2            study_directions
3       study_user_attributes
4     study_system_attributes
5                      trials
6       trial_user_attributes
7     trial_system_attributes
8                trial_params
9                trial_values
10  trial_intermediate_values
11           trial_heartbeats
12            alembic_version


In [26]:
data = pd.read_sql_query("SELECT tr.trial_id, sd.study_name, val.value FROM trials tr LEFT JOIN studies sd on tr.study_id = sd.study_id LEFT JOIN trial_values val on val.trial_id = tr.trial_id", conn)

In [32]:
data.groupby("study_name").aggregate({"value": np.max}).sort_values("value", ascending=False)

/var/folders/m4/j_mwks0x04ncx2w9srg4t8qh0000gn/T/ipykernel_2772/2968359179.py:1: FutureWarning: The provided callable <function max at 0x104e05e40> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  data.groupby("study_name").aggregate({"value": np.max}).sort_values("value", ascending=False)


,value
study_name,
Volatility_Breakout_Double_Strategy_XAG,9047.541783
Volatility_Breakout_Double_Strategy_SP,5596.558484
Volatility_Breakout_Strategy_NQ,3968.447443
Volatility_Breakout_Double_Strategy_NQ,2860.874298
Volatility_Breakout_Double_Strategy_XAU,1569.884619
Volatility_Breakout_Strategy_SP,802.334897
Volatility_Breakout_Double_Strategy_AUD,330.360683
Moving_Average_Strategy_NQ,237.649659
Volatility_Breakout_Strategy_XAG,163.883662


In [129]:
def process_results(data, group = 1):
    group_cols = [x for x in data.columns if x not in ['selected_quarter','stats_mm','stats_mr','stats_fl']]
    df = (
        data.replace(np.inf, 0)
        .groupby(group_cols)
        .aggregate({
            "stats_mm": [np.sum, np.std],
            "stats_mr": [np.sum, np.std],
            "stats_fl": [np.sum, np.std]
        })
        .query(f"group_data == {group}")
    )
    df.columns = ['_'.join(col).strip() for col in df.columns]
    df["stats_mm_sum_corr"] = df["stats_mm_sum"] - df["stats_mm_std"]*4
    df["stats_mr_sum_corr"] = df["stats_mr_sum"] - df["stats_mr_std"]*4
    df["stats_fl_sum_corr"] = df["stats_fl_sum"] - df["stats_fl_std"]*4

    # df['max_sum_stats'] = df[['stats_mm_sum', 'stats_mr_sum', 'stats_fl_sum']].max(axis=1)
    df['max_sum_stats_corr'] = df[['stats_mm_sum_corr', 'stats_mr_sum_corr', 'stats_fl_sum_corr']].max(axis=1)
    return df.sort_values("stats_fl_sum_corr", ascending=False)

In [130]:
vb2_results = pd.read_csv("results/Volatility_Breakout_Double_Strategy/results.csv")
vb_results = pd.read_csv("results/Volatility_Breakout_Strategy/results.csv")
ma_results = pd.read_csv("results/Moving_Average_Strategy/results.csv")

In [131]:
vb2_gr1 = process_results(vb2_results,1)
vb2_gr2 = process_results(vb2_results,2)

vb_gr1 = process_results(vb_results,1)
vb_gr2 = process_results(vb_results,2)

ma_gr1 = process_results(ma_results,1)
ma_gr2 = process_results(ma_results,2)

/var/folders/m4/j_mwks0x04ncx2w9srg4t8qh0000gn/T/ipykernel_2772/544426164.py:6: FutureWarning: The provided callable <function sum at 0x104e05760> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  .aggregate({
/var/folders/m4/j_mwks0x04ncx2w9srg4t8qh0000gn/T/ipykernel_2772/544426164.py:6: FutureWarning: The provided callable <function std at 0x104e06980> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  .aggregate({
/var/folders/m4/j_mwks0x04ncx2w9srg4t8qh0000gn/T/ipykernel_2772/544426164.py:6: FutureWarning: The provided callable <function sum at 0x104e05760> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  .aggregate({
/var

# 2VB

## GR1

In [132]:
params = {x:y for x,y in vb2_gr1.query("instrument_name == 'SP'").head(5).reset_index().to_dict(orient="records")[0].items() if "stat" not in x}

In [141]:
df = vb2_gr1.query("instrument_name == 'SP'")
params = {x:y for x,y in df.reset_index().to_dict(orient="records")[0].items() if "stat" not in x}
query_conditions = []
for key, value in params.items():
    if isinstance(value, int):
        query_conditions.append(f"{key} == {value}")
    elif isinstance(value, float):
        query_conditions.append(f"{key} == {round(value, 5)}")
    elif isinstance(value, str):
        query_conditions.append(f"{key} == '{value}'")
query = " and ".join(query_conditions)

In [142]:
vb2_results.query(query)

,selected_quarter,fast_ma,ma_diff,signal_estimator,window_regime,treshold_regime,volat_param,m_exit,m_diff,instrument_name,group_data,p_val,tr_cost,stats_mm,stats_mr,stats_fl
9989,2022_Q1,72,62,median,48,0.16716,61,2.70505,1.36318,SP,1,50,12,3546.016742,-9.085911,6166.964411
9990,2022_Q3,72,62,median,48,0.16716,61,2.70505,1.36318,SP,1,50,12,0.538328,-1.182787,0.538328
9991,2022_Q4,72,62,median,48,0.16716,61,2.70505,1.36318,SP,1,50,12,2.408703,-3.421650,2.408703
9992,2023_Q2,72,62,median,48,0.16716,61,2.70505,1.36318,SP,1,50,12,0.000000,-1.870886,-2.607497
9993,2023_Q4,72,62,median,48,0.16716,61,2.70505,1.36318,SP,1,50,12,0.000000,0.000000,0.000000
9994,2024_Q1,72,62,median,48,0.16716,61,2.70505,1.36318,SP,1,50,12,-4.124578,10.130604,-4.193181
9995,2024_Q2,72,62,median,48,0.16716,61,2.70505,1.36318,SP,1,50,12,16.545859,-4.843227,15.982410


In [144]:
vb2_gr1.query("instrument_name == 'SP'").head(5)

,,,,,,,,,,,,stats_mm_sum,stats_mm_std,stats_mr_sum,stats_mr_std,stats_fl_sum,stats_fl_std,stats_mm_sum_corr,stats_mr_sum_corr,stats_fl_sum_corr,max_sum_stats_corr
fast_ma,ma_diff,signal_estimator,window_regime,treshold_regime,volat_param,m_exit,m_diff,instrument_name,group_data,p_val,tr_cost,,,,,,,,,,
72,62,median,48,0.16716,61,2.70505,1.36318,SP,1,50,12,3561.385053,1339.316259,-10.273857,5.918443,6179.093174,2330.138762,882.752536,-22.110742,1518.815651,1518.815651
70,62,median,48,0.13832,60,2.70040,1.48849,SP,1,50,12,3489.357163,1313.974397,-6.853518,5.364426,6057.027484,2286.013101,861.408368,-17.582370,1485.001282,1485.001282
72,60,median,51,0.19115,60,2.65975,1.44789,SP,1,50,12,3352.600578,1262.613045,-4.990264,5.544292,5819.778217,2196.944271,827.374487,-16.078849,1425.889675,1425.889675
73,57,median,48,0.15784,61,2.73591,1.22445,SP,1,50,12,3183.770368,1187.295561,-15.070101,4.055134,5508.320796,2067.432001,809.179245,-23.180370,1373.456795,1373.456795
56,65,median,32,0.23598,54,2.53402,1.74436,SP,1,50,12,5058.744391,1899.044794,10.428196,13.890657,5054.696770,1899.298636,1260.654804,-17.353119,1256.099498,1260.654804


In [87]:
vb2_gr1.query("instrument_name == 'NQ'").head(5)

stats_mm_sum  \
fast_ma ma_diff signal_estimator window_regime treshold_regime volat_param m_exit  m_diff  instrument_name group_data p_val tr_cost                 
54      71      mean             54            0.01686         72          2.08637 0.76097 NQ              1          20    12        3166.547176   
60      74      mean             53            0.01636         79          2.11369 0.79592 NQ              1          20    12        2574.840741   
        76      mean             50            0.00080         80          2.28469 0.70020 NQ              1          20    12        2359.317828   
53      61      mean             93            0.08141         68          1.95906 0.77728 NQ              1          20    12        3100.278750   
67      80      mean             37            0.02027         87          2.19893 0.69793 NQ              1          20    12        1130.933391   

                                                                                                                                     stats_mm_std  \
fast_ma ma_diff signal_estimator window_regime treshold_regime volat_param m_exit  m_diff  instrument_name group_data p_val tr_cost                 
54      71      mean             54            0.01686         72          2.08637 0.76097 NQ              1          20    12        1185.060970   
60      74      mean             53            0.01636         79          2.11369 0.79592 NQ              1          20    12         946.783232   
        76      mean             50            0.00080         80          2.28469 0.70020 NQ              1          20    12         863.503490   
53      61      mean             93            0.08141         68          1.95906 0.77728 NQ              1          20    12         905.863199   
67      80      mean             37            0.02027         87          2.19893 0.69793 NQ              1          20    12         345.137753   

                                                                                                                                     stats_mr_sum  \
fast_ma ma_diff signal_estimator window_regime treshold_regime volat_param m_exit  m_diff  instrument_name group_data p_val tr_cost                 
54      71      mean             54            0.01686         72          2.08637 0.76097 NQ              1          20    12         -29.120341   
60      74      mean             53            0.01636         79          2.11369 0.79592 NQ              1          20    12         -23.018721   
        76      mean             50            0.00080         80          2.28469 0.70020 NQ              1          20    12         -19.996238   
53      61      mean             93            0.08141         68          1.95906 0.77728 NQ              1          20    12         -39.854063   
67      80      mean             37            0.02027         87          2.19893 0.69793 NQ              1          20    12         -11.390028   

                                                                                                                                     stats_mr_std  \
fast_ma ma_diff signal_estimator window_regime treshold_regime volat_param m_exit  m_diff  instrument_name group_data p_val tr_cost                 
54      71      mean             54            0.01686         72          2.08637 0.76097 NQ              1          20    12           4.031330   
60      74      mean             53            0.01636         79          2.11369 0.79592 NQ              1          20    12           5.713327   
        76      mean             50            0.00080         80          2.28469 0.70020 NQ              1          20    12           5.945851   
53      61      mean             93            0.08141         68          1.95906 0.77728 NQ              1          20    12           4.808536   
67      80      mean             37            0.02027         87          2.19893 0.69793 NQ              

## GR2

In [90]:
vb2_gr2.query("instrument_name == 'XAG'").head(5)

,,,,,,,,,,,,stats_mm_sum,stats_mm_std,stats_mr_sum,stats_mr_std,stats_fl_sum,stats_fl_std,stats_mm_sum_corr,stats_mr_sum_corr,stats_fl_sum_corr,max_sum_stats_corr
fast_ma,ma_diff,signal_estimator,window_regime,treshold_regime,volat_param,m_exit,m_diff,instrument_name,group_data,p_val,tr_cost,,,,,,,,,,
51,89,mean,98,0.07867,72,1.79056,1.84808,XAG,2,5000,10,2576.298905,916.012742,-18.452072,3.328019,9975.525843,3711.936241,2118.292534,-20.116081,8119.557722,8119.557722
54,90,mean,97,0.01176,72,1.76829,1.94877,XAG,2,5000,10,1706.459091,589.864896,-17.881306,3.022937,6505.918313,2403.080540,1411.526643,-19.392775,5304.378043,5304.378043
55,93,mean,95,0.01548,72,1.89110,1.82745,XAG,2,5000,10,1664.204524,571.767245,-20.636731,3.024550,6341.518356,2338.568923,1378.320901,-22.149007,5172.233895,5172.233895
57,88,mean,99,0.05990,72,1.99069,1.98211,XAG,2,5000,10,1234.310999,360.191965,-20.236623,2.969420,6194.279001,2229.136808,1054.215016,-21.721333,5079.710597,5079.710597
54,92,mean,100,0.01639,72,1.89033,1.96458,XAG,2,5000,10,1646.499759,560.426227,-18.713060,3.012391,6218.553483,2287.586343,1366.286645,-20.219255,5074.760312,5074.760312


In [96]:
vb2_gr2.query("instrument_name == 'XAU'").head(5)

stats_mm_sum  \
fast_ma ma_diff signal_estimator window_regime treshold_regime volat_param m_exit  m_diff  instrument_name group_data p_val tr_cost                 
48      54      median           92            0.39871         64          0.70194 1.31992 XAU             2          100   15        1730.015665   
50      54      median           18            0.50058         75          0.26333 1.71345 XAU             2          100   15        1324.487377   
53      59      median           19            0.59334         74          0.18217 1.95395 XAU             2          100   15        1289.811089   
        50      median           27            0.52575         75          0.21360 1.75532 XAU             2          100   15        1409.452667   
50      54      median           32            0.53571         73          0.12458 1.93961 XAU             2          100   15        1409.006928   

                                                                                                                                     stats_mm_std  \
fast_ma ma_diff signal_estimator window_regime treshold_regime volat_param m_exit  m_diff  instrument_name group_data p_val tr_cost                 
48      54      median           92            0.39871         64          0.70194 1.31992 XAU             2          100   15         640.524186   
50      54      median           18            0.50058         75          0.26333 1.71345 XAU             2          100   15         500.609173   
53      59      median           19            0.59334         74          0.18217 1.95395 XAU             2          100   15         527.334918   
        50      median           27            0.52575         75          0.21360 1.75532 XAU             2          100   15         532.723034   
50      54      median           32            0.53571         73          0.12458 1.93961 XAU             2          100   15         532.751139   

                                                                                                                                     stats_mr_sum  \
fast_ma ma_diff signal_estimator window_regime treshold_regime volat_param m_exit  m_diff  instrument_name group_data p_val tr_cost                 
48      54      median           92            0.39871         64          0.70194 1.31992 XAU             2          100   15         -12.515703   
50      54      median           18            0.50058         75          0.26333 1.71345 XAU             2          100   15          -4.719953   
53      59      median           19            0.59334         74          0.18217 1.95395 XAU             2          100   15          -1.318753   
        50      median           27            0.52575         75          0.21360 1.75532 XAU             2          100   15          -4.837716   
50      54      median           32            0.53571         73          0.12458 1.93961 XAU             2          100   15          -4.837716   

                                                                                                                                     stats_mr_std  \
fast_ma ma_diff signal_estimator window_regime treshold_regime volat_param m_exit  m_diff  instrument_name group_data p_val tr_cost                 
48      54      median           92            0.39871         64          0.70194 1.31992 XAU             2          100   15           2.076333   
50      54      median           18            0.50058         75          0.26333 1.71345 XAU             2          100   15           1.783975   
53      59      median           19            0.59334         74          0.18217 1.95395 XAU             2          100   15           2.339142   
        50      median           27            0.52575         75          0.21360 1.75532 XAU             2          100   15           1.828485   
50      54      median           32            0.53571         73          0.12458 1.93961 XAU             

## VB

## GR1

In [97]:
vb_gr1.query("instrument_name == 'SP'").head(5)

,,,,,,,,,,,stats_mm_sum,stats_mm_std,stats_mr_sum,stats_mr_std,stats_fl_sum,stats_fl_std,stats_mm_sum_corr,stats_mr_sum_corr,stats_fl_sum_corr,max_sum_stats_corr
fast_ma,ma_diff,signal_estimator,window_regime,treshold_regime,volat_param,m_,instrument_name,group_data,p_val,tr_cost,,,,,,,,,,
39,44,mean,68,0.65503,68,1.41443,SP,1,50,12,1334.319828,483.593785,-25.762721,3.306332,881.778228,317.773326,1092.522936,-27.415887,722.891565,1092.522936
36,48,median,83,0.35482,15,2.65307,SP,1,50,12,277.108433,60.201942,-61.324918,4.563078,268.779106,60.572255,247.007462,-63.606457,238.492978,247.007462
38,49,median,33,0.17176,15,2.63411,SP,1,50,12,260.592866,54.626264,-64.505037,4.764234,257.751782,55.916376,233.279734,-66.887154,229.793595,233.279734
36,48,median,92,0.41857,15,2.71111,SP,1,50,12,269.143769,62.939237,-58.041029,4.844428,259.535796,63.275547,237.674150,-60.463243,227.898022,237.674150
38,50,median,77,0.43895,16,2.83240,SP,1,50,12,254.492393,60.357712,-61.335481,4.227592,252.709485,60.633647,224.313537,-63.449277,222.392662,224.313537


In [98]:
vb_gr1.query("instrument_name == 'NQ'").head(5)

stats_mm_sum  \
fast_ma ma_diff signal_estimator window_regime treshold_regime volat_param m_      instrument_name group_data p_val tr_cost                 
57      58      mean             41            0.66677         69          2.68167 NQ              1          20    12        3622.183253   
        59      mean             41            0.50270         70          2.62781 NQ              1          20    12        3821.587220   
54      58      mean             42            0.67346         68          2.63429 NQ              1          20    12        2471.126905   
57      57      mean             40            0.64827         68          2.66573 NQ              1          20    12        2127.538199   
                                               0.66324         68          2.66351 NQ              1          20    12        2127.440003   

                                                                                                                             stats_mm_std  \
fast_ma ma_diff signal_estimator window_regime treshold_regime volat_param m_      instrument_name group_data p_val tr_cost                 
57      58      mean             41            0.66677         69          2.68167 NQ              1          20    12        1262.877839   
        59      mean             41            0.50270         70          2.62781 NQ              1          20    12        1317.241286   
54      58      mean             42            0.67346         68          2.63429 NQ              1          20    12         734.602856   
57      57      mean             40            0.64827         68          2.66573 NQ              1          20    12         599.417219   
                                               0.66324         68          2.66351 NQ              1          20    12         599.425541   

                                                                                                                             stats_mr_sum  \
fast_ma ma_diff signal_estimator window_regime treshold_regime volat_param m_      instrument_name group_data p_val tr_cost                 
57      58      mean             41            0.66677         69          2.68167 NQ              1          20    12         -36.075071   
        59      mean             41            0.50270         70          2.62781 NQ              1          20    12         -33.606583   
54      58      mean             42            0.67346         68          2.63429 NQ              1          20    12         -38.597151   
57      57      mean             40            0.64827         68          2.66573 NQ              1          20    12         -35.618419   
                                               0.66324         68          2.66351 NQ              1          20    12         -35.593048   

                                                                                                                             stats_mr_std  \
fast_ma ma_diff signal_estimator window_regime treshold_regime volat_param m_      instrument_name group_data p_val tr_cost                 
57      58      mean             41            0.66677         69          2.68167 NQ              1          20    12           4.973494   
        59      mean             41            0.50270         70          2.62781 NQ              1          20    12           5.340510   
54      58      mean             42            0.67346         68          2.63429 NQ              1          20    12           4.745652   
57      57      mean             40            0.64827         68          2.66573 NQ              1          20    12           5.119391   
                                               0.66324         68          2.66351 NQ              1          20    12           5.124698   

                                                                                                                             stats_fl_sum  \
fast_ma ma_diff signal_estimator

## GR2

In [99]:
vb_gr2.query("instrument_name == 'XAG'").head(5)

stats_mm_sum  \
fast_ma ma_diff signal_estimator window_regime treshold_regime volat_param m_      instrument_name group_data p_val tr_cost                 
59      8       median           77            0.18152         28          1.29500 XAG             2          5000  10         202.112547   
56      9       mean             71            0.33992         29          1.26913 XAG             2          5000  10         173.024567   
                                 72            0.33177         28          1.25641 XAG             2          5000  10         172.786796   
                                 71            0.32508         29          1.28987 XAG             2          5000  10         169.268171   
59      8       median           78            0.36401         28          1.26730 XAG             2          5000  10         174.062422   

                                                                                                                             stats_mm_std  \
fast_ma ma_diff signal_estimator window_regime treshold_regime volat_param m_      instrument_name group_data p_val tr_cost                 
59      8       median           77            0.18152         28          1.29500 XAG             2          5000  10          45.875541   
56      9       mean             71            0.33992         29          1.26913 XAG             2          5000  10          76.581821   
                                 72            0.33177         28          1.25641 XAG             2          5000  10          76.449092   
                                 71            0.32508         29          1.28987 XAG             2          5000  10          76.579976   
59      8       median           78            0.36401         28          1.26730 XAG             2          5000  10          39.058347   

                                                                                                                             stats_mr_sum  \
fast_ma ma_diff signal_estimator window_regime treshold_regime volat_param m_      instrument_name group_data p_val tr_cost                 
59      8       median           77            0.18152         28          1.29500 XAG             2          5000  10         -14.459125   
56      9       mean             71            0.33992         29          1.26913 XAG             2          5000  10          47.135911   
                                 72            0.33177         28          1.25641 XAG             2          5000  10          51.187280   
                                 71            0.32508         29          1.28987 XAG             2          5000  10          52.484619   
59      8       median           78            0.36401         28          1.26730 XAG             2          5000  10         -13.480239   

                                                                                                                             stats_mr_std  \
fast_ma ma_diff signal_estimator window_regime treshold_regime volat_param m_      instrument_name group_data p_val tr_cost                 
59      8       median           77            0.18152         28          1.29500 XAG             2          5000  10          10.098961   
56      9       mean             71            0.33992         29          1.26913 XAG             2          5000  10          17.173396   
                                 72            0.33177         28          1.25641 XAG             2          5000  10          19.594028   
                                 71            0.32508         29          1.28987 XAG             2          5000  10          16.870257   
59      8       median           78            0.36401         28          1.26730 XAG             2          5000  10          10.047042   

                                                                                                                             stats_fl_sum  \
fast_ma ma_diff signal_estimator

In [100]:
vb_gr2.query("instrument_name == 'XAU'").head(5)

,,,,,,,,,,,stats_mm_sum,stats_mm_std,stats_mr_sum,stats_mr_std,stats_fl_sum,stats_fl_std,stats_mm_sum_corr,stats_mr_sum_corr,stats_fl_sum_corr,max_sum_stats_corr
fast_ma,ma_diff,signal_estimator,window_regime,treshold_regime,volat_param,m_,instrument_name,group_data,p_val,tr_cost,,,,,,,,,,
33,55,median,94,0.04843,11,0.47290,XAU,2,100,15,167.972290,31.849951,-56.592853,4.134983,149.678194,31.369087,152.047314,-58.660344,133.993650,152.047314
31,57,median,96,0.42679,9,0.53374,XAU,2,100,15,155.701073,27.774240,-56.453262,4.232992,135.194792,26.696258,141.813953,-58.569758,121.846663,141.813953
33,54,median,98,0.47962,11,0.48399,XAU,2,100,15,155.033068,30.557190,-55.631922,4.090568,136.780373,30.107223,139.754473,-57.677206,121.726761,139.754473
31,56,median,96,0.04157,11,0.39932,XAU,2,100,15,158.772483,28.901782,-53.479338,5.000794,132.641125,26.389606,144.321593,-55.979735,119.446321,144.321593
33,57,median,98,0.04339,11,0.40093,XAU,2,100,15,156.435324,31.548562,-55.894580,4.073516,134.491874,30.408736,140.661043,-57.931338,119.287506,140.661043


# MA

## GR1

In [105]:
ma_gr1.query("instrument_name == 'SP'").head(5)

stats_mm_sum  \
fast_ma ma_diff signal_estimator window_regime treshold_regime instrument_name group_data p_val tr_cost                 
63      72      mean             84            0.21067         SP              1          50    12         155.630930   
                                 100           0.64932         SP              1          50    12         155.630930   
                                 96            0.68224         SP              1          50    12         155.630930   
                                 95            0.68540         SP              1          50    12         155.630930   
        70      mean             86            0.68641         SP              1          50    12         141.502136   

                                                                                                         stats_mm_std  \
fast_ma ma_diff signal_estimator window_regime treshold_regime instrument_name group_data p_val tr_cost                 
63      72      mean             84            0.21067         SP              1          50    12          62.831271   
                                 100           0.64932         SP              1          50    12          62.831271   
                                 96            0.68224         SP              1          50    12          62.831271   
                                 95            0.68540         SP              1          50    12          62.831271   
        70      mean             86            0.68641         SP              1          50    12          59.559217   

                                                                                                         stats_mr_sum  \
fast_ma ma_diff signal_estimator window_regime treshold_regime instrument_name group_data p_val tr_cost                 
63      72      mean             84            0.21067         SP              1          50    12         -39.398574   
                                 100           0.64932         SP              1          50    12         -39.398574   
                                 96            0.68224         SP              1          50    12         -39.398574   
                                 95            0.68540         SP              1          50    12         -39.398574   
        70      mean             86            0.68641         SP              1          50    12         -36.806112   

                                                                                                         stats_mr_std  \
fast_ma ma_diff signal_estimator window_regime treshold_regime instrument_name group_data p_val tr_cost                 
63      72      mean             84            0.21067         SP              1          50    12           4.324925   
                                 100           0.64932         SP              1          50    12           4.324925   
                                 96            0.68224         SP              1          50    12           4.324925   
                                 95            0.68540         SP              1          50    12           4.324925   
        70      mean             86            0.68641         SP              1          50    12           4.756498   

                                                                                                         stats_fl_sum  \
fast_ma ma_diff signal_estimator window_regime treshold_regime instrument_name group_data p_val tr_cost                 
63      72      mean             84            0.21067         SP              1          50    12         151.619698   
                                 100           0.64932         SP              1          50    12         150.250490   
                                 96            0.68224         SP              1          50    12         148.943226   
                                 95            0.68540         SP              1          50    12         1

In [106]:
ma_gr1.query("instrument_name == 'NQ'").head(5)

stats_mm_sum  \
fast_ma ma_diff signal_estimator window_regime treshold_regime instrument_name group_data p_val tr_cost                 
70      91      mean             93            0.01383         NQ              1          20    12         262.877749   
                                 99            0.11259         NQ              1          20    12         262.877749   
                                 47            0.10671         NQ              1          20    12         262.877749   
                                 35            0.07717         NQ              1          20    12         262.877749   
71      87      mean             86            0.04758         NQ              1          20    12         255.346684   

                                                                                                         stats_mm_std  \
fast_ma ma_diff signal_estimator window_regime treshold_regime instrument_name group_data p_val tr_cost                 
70      91      mean             93            0.01383         NQ              1          20    12          95.921127   
                                 99            0.11259         NQ              1          20    12          95.921127   
                                 47            0.10671         NQ              1          20    12          95.921127   
                                 35            0.07717         NQ              1          20    12          95.921127   
71      87      mean             86            0.04758         NQ              1          20    12          94.989643   

                                                                                                         stats_mr_sum  \
fast_ma ma_diff signal_estimator window_regime treshold_regime instrument_name group_data p_val tr_cost                 
70      91      mean             93            0.01383         NQ              1          20    12           0.496528   
                                 99            0.11259         NQ              1          20    12           0.496528   
                                 47            0.10671         NQ              1          20    12           0.496528   
                                 35            0.07717         NQ              1          20    12           0.496528   
71      87      mean             86            0.04758         NQ              1          20    12          -2.806749   

                                                                                                         stats_mr_std  \
fast_ma ma_diff signal_estimator window_regime treshold_regime instrument_name group_data p_val tr_cost                 
70      91      mean             93            0.01383         NQ              1          20    12          13.819655   
                                 99            0.11259         NQ              1          20    12          13.819655   
                                 47            0.10671         NQ              1          20    12          13.819655   
                                 35            0.07717         NQ              1          20    12          13.819655   
71      87      mean             86            0.04758         NQ              1          20    12          13.647015   

                                                                                                         stats_fl_sum  \
fast_ma ma_diff signal_estimator window_regime treshold_regime instrument_name group_data p_val tr_cost                 
70      91      mean             93            0.01383         NQ              1          20    12         261.594424   
                                 99            0.11259         NQ              1          20    12         261.574669   
                                 47            0.10671         NQ              1          20    12         260.823421   
                                 35            0.07717         NQ              1          20    12         2

## GR2

In [108]:
ma_gr2.groupby("instrument_name").max().sort_values("max_sum_stats_corr", ascending=False)

,stats_mm_sum,stats_mm_std,stats_mr_sum,stats_mr_std,stats_fl_sum,stats_fl_std,stats_mm_sum_corr,stats_mr_sum_corr,stats_fl_sum_corr,max_sum_stats_corr
instrument_name,,,,,,,,,,
XAG,87.507750,39.964567,66.978377,36.203351,85.875152,41.199370,67.525467,48.876701,65.275466,67.525467
XAU,61.057628,23.933957,-2.692037,7.496773,72.467240,24.411313,49.090649,-6.440423,60.621671,60.621671
AUD,6.276856,8.212254,-3.079029,8.318272,6.590039,8.328840,2.170729,-5.959205,2.425619,2.425619
CAD,-11.550902,2.327846,-2.289773,2.456230,-11.442626,2.380426,-12.457133,-2.563580,-12.342026,-2.563580


In [107]:
ma_gr2.query("instrument_name == 'XAG'").head(5)

stats_mm_sum  \
fast_ma ma_diff signal_estimator window_regime treshold_regime instrument_name group_data p_val tr_cost                 
48      84      median           19            0.05383         XAG             2          5000  10          87.507750   
53      79      median           32            0.01569         XAG             2          5000  10          60.910348   
54      82      median           31            0.01432         XAG             2          5000  10          62.102461   
52      78      median           24            0.04258         XAG             2          5000  10          66.164253   
                                 22            0.03698         XAG             2          5000  10          66.164253   

                                                                                                         stats_mm_std  \
fast_ma ma_diff signal_estimator window_regime treshold_regime instrument_name group_data p_val tr_cost                 
48      84      median           19            0.05383         XAG             2          5000  10          39.964567   
53      79      median           32            0.01569         XAG             2          5000  10          29.780351   
54      82      median           31            0.01432         XAG             2          5000  10          29.138174   
52      78      median           24            0.04258         XAG             2          5000  10          30.934600   
                                 22            0.03698         XAG             2          5000  10          30.934600   

                                                                                                         stats_mr_sum  \
fast_ma ma_diff signal_estimator window_regime treshold_regime instrument_name group_data p_val tr_cost                 
48      84      median           19            0.05383         XAG             2          5000  10          34.249614   
53      79      median           32            0.01569         XAG             2          5000  10          -0.676093   
54      82      median           31            0.01432         XAG             2          5000  10          -2.200265   
52      78      median           24            0.04258         XAG             2          5000  10          14.345825   
                                 22            0.03698         XAG             2          5000  10          14.345825   

                                                                                                         stats_mr_std  \
fast_ma ma_diff signal_estimator window_regime treshold_regime instrument_name group_data p_val tr_cost                 
48      84      median           19            0.05383         XAG             2          5000  10          18.814286   
53      79      median           32            0.01569         XAG             2          5000  10           8.825874   
54      82      median           31            0.01432         XAG             2          5000  10           8.725126   
52      78      median           24            0.04258         XAG             2          5000  10          15.057501   
                                 22            0.03698         XAG             2          5000  10          15.057501   

                                                                                                         stats_fl_sum  \
fast_ma ma_diff signal_estimator window_regime treshold_regime instrument_name group_data p_val tr_cost                 
48      84      median           19            0.05383         XAG             2          5000  10          85.875152   
53      79      median           32            0.01569         XAG             2          5000  10          82.016891   
54      82      median           31            0.01432         XAG             2          5000  10          79.248221   
52      78      median           24            0.04258         XAG             2          5000  10          

In [109]:
ma_gr2.query("instrument_name == 'XAU'").head(5)

stats_mm_sum  \
fast_ma ma_diff signal_estimator window_regime treshold_regime instrument_name group_data p_val tr_cost                 
45      41      mean             88            0.68445         XAU             2          100   15          40.520259   
        42      mean             91            0.69684         XAU             2          100   15          43.344755   
                                 90            0.63986         XAU             2          100   15          43.344755   
                                 91            0.64453         XAU             2          100   15          43.344755   
                                 90            0.61975         XAU             2          100   15          43.344755   

                                                                                                         stats_mm_std  \
fast_ma ma_diff signal_estimator window_regime treshold_regime instrument_name group_data p_val tr_cost                 
45      41      mean             88            0.68445         XAU             2          100   15          17.899310   
        42      mean             91            0.69684         XAU             2          100   15          17.571506   
                                 90            0.63986         XAU             2          100   15          17.571506   
                                 91            0.64453         XAU             2          100   15          17.571506   
                                 90            0.61975         XAU             2          100   15          17.571506   

                                                                                                         stats_mr_sum  \
fast_ma ma_diff signal_estimator window_regime treshold_regime instrument_name group_data p_val tr_cost                 
45      41      mean             88            0.68445         XAU             2          100   15         -34.500658   
        42      mean             91            0.69684         XAU             2          100   15         -31.909463   
                                 90            0.63986         XAU             2          100   15         -31.909463   
                                 91            0.64453         XAU             2          100   15         -31.909463   
                                 90            0.61975         XAU             2          100   15         -31.909463   

                                                                                                         stats_mr_std  \
fast_ma ma_diff signal_estimator window_regime treshold_regime instrument_name group_data p_val tr_cost                 
45      41      mean             88            0.68445         XAU             2          100   15           5.238048   
        42      mean             91            0.69684         XAU             2          100   15           5.928562   
                                 90            0.63986         XAU             2          100   15           5.928562   
                                 91            0.64453         XAU             2          100   15           5.928562   
                                 90            0.61975         XAU             2          100   15           5.928562   

                                                                                                         stats_fl_sum  \
fast_ma ma_diff signal_estimator window_regime treshold_regime instrument_name group_data p_val tr_cost                 
45      41      mean             88            0.68445         XAU             2          100   15          72.467240   
        42      mean             91            0.69684         XAU             2          100   15          71.795379   
                                 90            0.63986         XAU             2          100   15          71.200879   
                                 91            0.64453         XAU             2          100   15          